In [80]:
import os
import numpy as np
import pickle
import networkx as nx
from tqdm import tqdm
import statsmodels.stats.proportion as smp

HOME_DIR = '/home/DC_replication'
OUTPUT_DIR = './output'

In [81]:
target_subject = 'mnist'
total_model_num = 20

### Load ki files

In [82]:
kis_dmpp, kis_dc = {}, {}


# import ki files (dm++)
dmpp_ki_dir = f'{OUTPUT_DIR}/ki/'    
ki_files = os.listdir(dmpp_ki_dir)
for ki_file in ki_files:
    if ki_file.endswith('ki.npy'):
        ki = np.load(os.path.join(dmpp_ki_dir, ki_file))
        kis_dmpp[ki_file] = ki

# import ki files (dc)
dc_ki_dir = f'{HOME_DIR}/Data/inputs_killability/{target_subject}'    
for ki_file in os.listdir(dc_ki_dir):
    if ki_file.startswith(target_subject) and ki_file.endswith('_ki.npy'):
        ki = np.load(os.path.join(dc_ki_dir, ki_file))        
        kis_dc[ki_file] = ki


In [83]:
print(kis_dmpp['mnist_op:4_mutant:11_ki.npy'].shape, len(kis_dmpp))
print(kis_dc['mnist_change_epochs_mutated0_MP_False_8_ki.npy'].shape, len(kis_dc))

(20, 60000) 92
(20, 60000) 218


### generate_unstable_inputs_map

In [89]:
def get_confidence_intervals():
    confidence_intervals = []

    for i in range(0, total_model_num+1):
        (low, high) = smp.proportion_confint(i, total_model_num, alpha=0.10, method='wilson')
        confidence_intervals.append((low, high))

    return confidence_intervals

In [90]:
confidence_intervals = get_confidence_intervals()
input_dict = dict()
input_dict_file = os.path.join(OUTPUT_DIR, "unstable_input_dict.pickle")

if not os.path.exists(input_dict_file):
    for mut1, ki1 in tqdm(kis_dc.items()):
        killing_probabilities1 = np.sum(ki1, axis=0) / len(ki1)
        input_dict[mut1] = {}
        for mut2, ki2 in kis_dmpp.items():
            overlap_num = 0
            indices_to_delete = []
            killing_probabilities2 = np.sum(ki2, axis=0) / len(ki2)

            for i in range(0, len(killing_probabilities1)):
                num_killed1 = int(killing_probabilities1[i] * total_model_num)
                num_killed2 = int(killing_probabilities2[i] * total_model_num)

                interval1 = confidence_intervals[num_killed1]
                interval2 = confidence_intervals[num_killed2]

                if interval1[0] <= interval2[0] <= interval1[1] or interval2[0] <= interval1[0] <= interval2[1]:
                    indices_to_delete.append(i)
                    overlap_num = overlap_num + 1

            input_dict[mut1][mut2] = indices_to_delete

    with open(input_dict_file, 'wb') as f:
        pickle.dump(input_dict, f, pickle.HIGHEST_PROTOCOL)

100%|██████████| 218/218 [49:19<00:00, 13.58s/it]


In [91]:
input_dict = dict()
input_dict_file = os.path.join(OUTPUT_DIR, "unstable_input_dict2.pickle")

if not os.path.exists(input_dict_file):
    for mut1, ki1 in tqdm(kis_dmpp.items()):
        killing_probabilities1 = np.sum(ki1, axis=0) / len(ki1)
        input_dict[mut1] = {}
        for mut2, ki2 in kis_dc.items():
            overlap_num = 0
            indices_to_delete = []
            killing_probabilities2 = np.sum(ki2, axis=0) / len(ki2)

            for i in range(0, len(killing_probabilities1)):
                num_killed1 = int(killing_probabilities1[i] * total_model_num)
                num_killed2 = int(killing_probabilities2[i] * total_model_num)

                interval1 = confidence_intervals[num_killed1]
                interval2 = confidence_intervals[num_killed2]

                if interval1[0] <= interval2[0] <= interval1[1] or interval2[0] <= interval1[0] <= interval2[1]:
                    indices_to_delete.append(i)
                    overlap_num = overlap_num + 1

            input_dict[mut1][mut2] = indices_to_delete

    with open(input_dict_file, 'wb') as f:
        pickle.dump(input_dict, f, pickle.HIGHEST_PROTOCOL)

100%|██████████| 92/92 [49:29<00:00, 32.27s/it]


### analyse_redundancy (to see whether DC mutants subsume DM++ mutants)
Here I used "unstable_input_dict"

In [93]:
input_dict_file = os.path.join(OUTPUT_DIR, "unstable_input_dict.pickle")
with open(input_dict_file, 'rb') as f:
    unstable_inputs_dict = pickle.load(f)

In [94]:
g = nx.DiGraph()
threshold = 100

for mut1, ki1 in kis_dc.items():
    killing_probabilities1 = np.sum(ki1, axis=0) / len(ki1)
    if mut1 not in g:
        g.add_node(mut1)

    for mut2, ki2 in kis_dmpp.items():
        if mut2 not in unstable_inputs_dict[mut1]:
            continue
        killing_probabilities2 = np.sum(ki2, axis=0) / len(ki2)
        indices_to_delete = unstable_inputs_dict[mut1][mut2]

        if len(indices_to_delete) < len(killing_probabilities1):
            killing_probabilities1_copy = np.delete(killing_probabilities1, indices_to_delete)
            killing_probabilities2_copy = np.delete(killing_probabilities2, indices_to_delete)

            if len(killing_probabilities1_copy) != 0 and len(killing_probabilities2_copy) != 0:
                comparison_array = np.less_equal(killing_probabilities1_copy, killing_probabilities2_copy)
                num_true = len(np.argwhere(comparison_array == True))
                # print(num_true)
                sc_low, sc_high = smp.proportion_confint(num_true, len(killing_probabilities1_copy), alpha=0.10, method='wilson')
                error = (sc_high - sc_low) / 2

                if error < 0.05 and num_true >= (threshold / 100) * len(killing_probabilities1_copy):
                    g.add_edge(mut1, mut2)

nodes = np.sort(g.nodes())

redundant_nodes = []
non_redundant_nodes = []
for node in nodes:
    if g.in_degree(node) == 0:
        non_redundant_nodes.append(str(node))
    else:
        redundant_nodes.append(str(node))

print(len(non_redundant_nodes))
print(len(redundant_nodes))


218
50


### analyse_redundancy (to see whether DM++ mutants subsume DC mutants)
Here I used "unstable_input_dict2"

In [95]:
input_dict_file = os.path.join(OUTPUT_DIR, "unstable_input_dict2.pickle")
with open(input_dict_file, 'rb') as f:
    unstable_inputs_dict = pickle.load(f)

In [96]:
g = nx.DiGraph()
threshold = 100

for mut1, ki1 in kis_dmpp.items():
    killing_probabilities1 = np.sum(ki1, axis=0) / len(ki1)
    if mut1 not in g:
        g.add_node(mut1)

    for mut2, ki2 in kis_dc.items():
        if mut2 not in unstable_inputs_dict[mut1]:
            continue
        killing_probabilities2 = np.sum(ki2, axis=0) / len(ki2)
        indices_to_delete = unstable_inputs_dict[mut1][mut2]

        if len(indices_to_delete) < len(killing_probabilities1):
            killing_probabilities1_copy = np.delete(killing_probabilities1, indices_to_delete)
            killing_probabilities2_copy = np.delete(killing_probabilities2, indices_to_delete)

            if len(killing_probabilities1_copy) != 0 and len(killing_probabilities2_copy) != 0:
                comparison_array = np.less_equal(killing_probabilities1_copy, killing_probabilities2_copy)
                num_true = len(np.argwhere(comparison_array == True))                
                sc_low, sc_high = smp.proportion_confint(num_true, len(killing_probabilities1_copy), alpha=0.10, method='wilson')
                error = (sc_high - sc_low) / 2

                if error < 0.05 and num_true >= (threshold / 100) * len(killing_probabilities1_copy):
                    g.add_edge(mut1, mut2)

nodes = np.sort(g.nodes())

redundant_nodes = []
non_redundant_nodes = []
for node in nodes:
    if g.in_degree(node) == 0:
        non_redundant_nodes.append(str(node))
    else:
        redundant_nodes.append(str(node))

print(len(non_redundant_nodes))
print(len(redundant_nodes))

92
208


In [98]:
for node in nodes:
    if g.in_degree(node) != 0:
        print(node, g.in_degree(node))

mnist_add_noise_mutated0_MP_100_ki.npy 14
mnist_add_noise_mutated0_MP_25.0_ki.npy 4
mnist_add_noise_mutated0_MP_31.25_ki.npy 5
mnist_add_noise_mutated0_MP_34.38_ki.npy 6
mnist_add_noise_mutated0_MP_37.5_ki.npy 4
mnist_add_noise_mutated0_MP_40.62_ki.npy 5
mnist_add_noise_mutated0_MP_43.75_ki.npy 7
mnist_add_noise_mutated0_MP_46.88_ki.npy 12
mnist_add_noise_mutated0_MP_50.0_ki.npy 11
mnist_add_noise_mutated0_MP_56.25_ki.npy 9
mnist_add_noise_mutated0_MP_59.38_ki.npy 8
mnist_add_noise_mutated0_MP_62.5_ki.npy 12
mnist_add_noise_mutated0_MP_68.75_ki.npy 14
mnist_add_noise_mutated0_MP_71.88_ki.npy 13
mnist_add_noise_mutated0_MP_75.0_ki.npy 12
mnist_add_noise_mutated0_MP_87.5_ki.npy 17
mnist_add_noise_mutated0_MP_90.62_ki.npy 17
mnist_add_noise_mutated0_MP_93.75_ki.npy 17
mnist_add_noise_mutated0_MP_96.88_ki.npy 13
mnist_add_weights_regularisation_mutated0_MP_l1_0_ki.npy 13
mnist_add_weights_regularisation_mutated0_MP_l1_l2_0_ki.npy 14
mnist_add_weights_regularisation_mutated0_MP_l2_0_ki.npy 

### Cheking one example mutant

In [111]:
num_inputs = 60000

example_mutant = 'mnist_unbalance_train_data_mutated0_MP_96.88_ki.npy'
ki_subsumed = kis_dc[example_mutant]
kp_subsumed = np.sum(ki_subsumed, axis=0) / len(ki_subsumed)

for subsuming_mutant, _ in g.in_edges(example_mutant):
    if subsuming_mutant not in kis_dmpp:
        continue

    print("subsuming: ", subsuming_mutant)
    ki_subsuming = kis_dmpp[subsuming_mutant]
    kp_subsuming = np.sum(ki_subsuming, axis=0) / len(ki_subsuming)

    deleted = unstable_inputs_dict[subsuming_mutant][example_mutant]    
    not_deleted = sorted(list(set(range(num_inputs)) - set(deleted)))
    for i in not_deleted:        
        print(i, kp_subsumed[i], kp_subsuming[i])        
    break


# for mut1, ki1 in kis_dmpp.items():
#     killing_probabilities1 = np.sum(ki1, axis=0) / len(ki1)    

#     for mut2, ki2 in kis_dc.items():


subsuming:  mnist_op:0_mutant:1_ki.npy
22 0.4 0.0
48 1.0 0.0
54 0.95 0.05
154 0.85 0.0
160 0.6 0.0
172 0.9 0.05
240 0.3 0.0
264 1.0 0.0
282 0.8 0.0
304 1.0 0.0
346 0.95 0.0
362 0.25 0.0
404 1.0 0.0
418 0.75 0.05
424 0.5 0.0
456 0.3 0.0
482 1.0 0.0
485 0.3 0.0
517 0.9 0.0
528 0.85 0.0
589 0.35 0.0
602 0.35 0.0
626 0.7 0.0
631 0.25 0.0
641 0.35 0.0
704 0.8 0.0
720 1.0 0.0
784 0.35 0.0
788 0.95 0.05
792 0.85 0.0
864 0.85 0.0
902 0.35 0.0
974 0.5 0.0
1066 0.5 0.0
1116 0.3 0.0
1120 0.9 0.0
1138 0.9 0.0
1207 0.45 0.0
1214 0.45 0.0
1219 0.35 0.0
1239 0.85 0.0
1282 0.9 0.0
1316 0.25 0.0
1352 1.0 0.15
1360 0.55 0.0
1364 0.9 0.0
1518 0.4 0.0
1522 0.85 0.0
1552 0.6 0.0
1598 0.95 0.0
1622 0.45 0.0
1674 0.95 0.0
1683 0.6 0.0
1758 0.75 0.0
1804 0.55 0.0
1875 0.9 0.0
1914 0.5 0.0
1925 0.3 0.0
1940 0.7 0.05
1948 0.85 0.05
1968 0.95 0.0
2030 0.75 0.0
2141 0.25 0.0
2150 0.7 0.0
2194 0.25 0.0
2231 0.7 0.0
2302 0.85 0.0
2385 0.6 0.0
2421 0.35 0.0
2425 0.55 0.0
2426 0.25 0.0
2438 0.8 0.0
2593 0.5 0.0
2598 